In [3]:
from database.team_stats import TeamStatsManager
import pandas as pd

In [4]:
file_path = 'data/jornadas/2025/result_season_1_week_1.xlsm'

Este bloque actualiza la tabla de resultados de equipos

In [65]:
# Lee team resultados. Tabla 1
df_resultado = pd.read_excel(file_path, sheet_name='Partido', skiprows=3, usecols='B:G')
df_resultado.columns = ['local', 'vs', 'away', 'local_goals', '-', 'away_goals']

# Encontramos el inicio de la tabla stats
idx_stats = df_resultado[df_resultado['vs'] == 'Equipo'].index
idx_stats = idx_stats[0] + 4

df_resultado = df_resultado.drop(['vs', '-'], axis=1)
df_resultado = df_resultado.dropna()
#to int
df_resultado['local_goals'] = df_resultado['local_goals'].astype(int)
df_resultado['away_goals'] = df_resultado['away_goals'].astype(int)
df_resultado

,local,away,local_goals,away_goals
0,Amarillo,Rojo,2,0
1,Amarillo,Azul,1,0
2,Amarillo,Rojo,0,1
3,Rojo,Azul,1,0
4,Rojo,Amarillo,1,2
5,Azul,Amarillo,1,2
6,Rojo,Amarillo,2,1
7,Rojo,Azul,1,0
8,Rojo,Amarillo,2,1
9,Azul,Amarillo,2,1


In [67]:
# Get data stats from results
df_stats = df_resultado.copy()
teams = df_stats['local'].unique()
# add columns
for team in teams:
    df_stats[team] = 0

df_stats['local_points'] = df_stats.apply(
    lambda x: 3 if x['local_goals'] > x['away_goals']
        else 1 if x['local_goals'] == x['away_goals'] else 0, axis=1)
df_stats['away_points'] = df_stats.apply(
    lambda x: 3 if x['away_goals'] > x['local_goals']
        else 1 if x['away_goals'] == x['local_goals'] else 0, axis=1)

# Rename
local_stats = df_stats[['local', 'local_goals', 'away_goals', 'local_points']].rename(
    columns={'local': 'team', 'local_goals': 'goals', 'away_goals': 'goals_against', 'local_points': 'points'})
away_stats = df_stats[['away', 'away_goals', 'local_goals', 'away_points']].rename(
    columns={'away': 'team', 'away_goals': 'goals', 'local_goals': 'goals_against', 'away_points': 'points'})

df_stats = pd.concat([local_stats, away_stats])
df_stats = df_stats.groupby('team').sum().reset_index()

df_stats


,team,goals,goals_against,points
0,Amarillo,12,12,13
1,Azul,3,8,3
2,Rojo,13,8,22


,Jugador,Equipo
0,Eddy,Azul
1,De Tovar,Azul
2,Emanuel,Azul
3,Chino,Azul
4,Fran,Azul
5,Christian,Azul
6,Carlos,Azul
7,Joao,Amarillo
8,Samuel Rojas,Amarillo
9,Jhon,Amarillo


In [42]:
# Get players stats
try:
        df_names = pd.read_excel(file_path, sheet_name='Registro', skiprows=1)
except FileNotFoundError:
    print(f"Error: The file {file_path} was not found.")

except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")


df_names.drop(columns=['Orden'], inplace=True)
df_names = df_names.iloc[:-3, :].copy()
df_names.columns = ['name', 'team']

try:
    df_player_stats = pd.read_excel(file_path, skiprows=3, sheet_name='Partido', usecols="L:O")
except Exception as e:
    print(f"An error occurred while reading the player stats: {e}")


df_player_stats.columns = ['name', 'team', 'goals', 'assists']
df_player_stats.dropna(how='all', inplace=True, subset=['goals', 'assists'])

df_stats = df_player_stats.merge(df_names, on=['name', 'team'], how='outer').copy()

# Maneja autogoles. Localiza palabra Autogol en la columna 'name'
autogol_indices = df_stats[df_stats['name'].str.startswith('Autogol')].index
for index in autogol_indices:
    print(f"Parece que hay un Autogol: {df_stats.loc[index, 'name']}")
    print("Selecciona el autor de este autogol: ")
    print(df_names)
    index_name = input("Selecciona un número de los anteriores")
    name = df_names.loc[int(index_name), 'name']
    df_stats.loc[df_stats['name'] == name, 'autogol'] = 1
df_stats.drop(autogol_indices, inplace=True)

df_stats.fillna(0, inplace=True)

# to int
df_stats['goals'] = df_stats['goals'].astype(int)
df_stats['assists'] = df_stats['assists'].astype(int)
df_stats['autogol'] = df_stats['autogol'].astype(int)

df_stats

Parece que hay un Autogol: Autogol Amarillo
Selecciona el autor de este autogol: 
            name      team
0           Eddy      Azul
1       De Tovar      Azul
2        Emanuel      Azul
3          Chino      Azul
4           Fran      Azul
5      Christian      Azul
6         Carlos      Azul
7           Joao  Amarillo
8   Samuel Rojas  Amarillo
9           Jhon  Amarillo
10       Ricardo  Amarillo
11    Luis Gocho  Amarillo
12       Vicente  Amarillo
13          Joel  Amarillo
14         Diego      Rojo
15   Andres Diaz      Rojo
16           Abu      Rojo
17        Imanol      Rojo
18   Andres Ruiz      Rojo
19         Bosch      Rojo
20         Pedro      Rojo


,name,team,goals,assists,autogol
0,Abu,Rojo,3,2,0
1,Andres Diaz,Rojo,2,1,0
2,Andres Ruiz,Rojo,1,2,0
4,Bosch,Rojo,0,4,0
5,Carlos,Azul,0,0,0
6,Chino,Azul,0,0,0
7,Christian,Azul,0,0,0
8,De Tovar,Azul,1,0,0
9,Diego,Rojo,5,1,0
10,Eddy,Azul,2,1,0


Este bloque actualiza la de estadísticas agregando los stats de la jornada desde el excel de jornada.
- name
- team
- goals
- assists

Se debe leer luego desde la app para recoger 
- media
- mvp
- votes
- total_votes 

 Actualizar

In [ ]:
# check if the week result already exists
update_week = True
if trm.get_week_result(year, season, week):
    update = input("The week result already exists. Do you want to update it? (y/n): ")
    if update.lower() == 'y':
        trm.delete_week_result(year, season, week)
    else:
        update_week = False

if update_week:
    for row in df_resultado.iterrows():
        trm.add_team_result(
            year=year,
            season=season,
            match_week=week,
            team_name=row[1]["Equipo"],
            goals=row[1]["A favor"],
            goals_against=row[1]["En contra"],
            points=row[1]["Puntos"]
    )

In [11]:
# Agregando las estadisticas de los jugadores
# Primero leemos el registro de jugadores
df_players = pd.read_excel(file_path, skiprows=1, nrows=28)
df_players = df_players.iloc[:, 1:].copy()
df_players.columns = ['Nombre', 'Equipo']
df_players.dropna(inplace=True, subset='Nombre', axis=0)

df_statistics = pd.read_excel(file_path, sheet_name='Partido', skiprows=3, usecols="L:O")
df_statistics.dropna(inplace=True, axis=0, how='all')
df_statistics.fillna(0, inplace=True)
df_statistics.drop(columns='Equipo', inplace=True)
df_statistics.columns = df_statistics.columns.str.strip()

df_excel = pd.merge(df_players, df_statistics, on='Nombre', how='outer')
df_excel.fillna(0, inplace=True)
df_excel.columns = ['name', 'team', 'goals', 'assists']


from database.players import PlayerManager
pm = PlayerManager()
fantasy_id_map = pm.get_fantasy_id()

df_excel['id'] = df_excel['name'].map(fantasy_id_map)
df_excel

c:\Users\Eddy.Tovar\Documents\Datos\personal\proyectos\fantasy_fulbo\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,name,team,goals,assists,id
0,Abu,Azul,2.0,1.0,1
1,Andres Diaz,Azul,0.0,0.0,4
2,Andres Freitas,Amarillo,0.0,1.0,5
3,Andres Ruiz,Azul,0.0,0.0,7
4,Arturo,Rojo,2.0,0.0,10
5,Bravo,Amarillo,0.0,0.0,14
6,Carlos,Negro,0.0,1.0,15
7,Chino,Rojo,1.0,0.0,16
8,De Tovar,Amarillo,0.0,0.0,19
9,Della Invitado,Rojo,0.0,1.0,65


Falta por agregar el código que escrapea desde la app. Lo he hecho en otro notebook para poder recuperar el histórico.